In [1]:
import pandas as pd
import csv
import numpy as np

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

seed = 42
torch.manual_seed(seed)

torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)

%matplotlib inline

use_cuda = torch.cuda.is_available()

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
                
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

def filterPair(p):
    return len(p[0]) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def getChars(item):
    return [element for element in item]

In [3]:
def get_data(file_range):
    print('File range:', list(file_range))
    data_list = []
    for i in file_range:
        index = str(i)
        if len(index) == 1:
            filename = 'output-0000{}-of-00100'.format(index)
        elif len(index) == 2:
            filename = 'output-000{}-of-00100'.format(index)
        else:
            raise ValueError('Wrong index')

        cur_data = pd.read_csv('../input/ru_with_types/' + filename, sep='\t', names=['class', 'before', 'after'],
                           quoting=csv.QUOTE_NONE, encoding='utf-8', dtype=str)

        if (cur_data.shape[0] > 1074563-10) and (cur_data.shape[0] < 1074563+10):
            print(filename)
        data_list.append(cur_data)
        print('Data shape for item {} is {}'.format(i,cur_data.shape))


    data_orig = pd.concat(data_list, axis=0)
    print('Overall data shape is {}'.format(data_orig.shape))
    
    return data_orig

In [5]:
data_dev = get_data(range(0,2))
data_learn = get_data(range(5,8))

File range: [0, 1]
Data shape for item 0 is (2973646, 3)
Data shape for item 1 is (2981069, 3)
Overall data shape is (5954715, 3)
File range: [5, 6, 7]
Data shape for item 5 is (2975471, 3)
Data shape for item 6 is (2980056, 3)
Data shape for item 7 is (2975255, 3)
Overall data shape is (8930782, 3)


In [6]:
data_train = pd.read_csv('../input/ru_train.csv', dtype = str)
#data_test = pd.read_csv('../input/ru_test.csv', dtype = str)
data_train.shape

(10574516, 5)

In [7]:
MAX_LENGTH = 100

In [8]:
def make_sample(data_learn, self_frac = 0.33, sil_frac = 1):

    data_nn = data_learn.copy()
    to_concat = []
    to_concat.append(data_nn[(data_nn.after != '<self>') & (data_nn.after != 'sil')])
    to_concat.append(data_nn[data_nn.after == '<self>'].sample(frac = self_frac))
    to_concat.append(data_nn[data_nn.after == 'sil'].sample(frac = sil_frac))

    data_nn = pd.concat(to_concat, axis=0)
    return data_nn

In [9]:
#data_nn = make_sample(data_learn)
#data_dev = make_sample(data_dev, self_frac=1.0)

In [10]:
#data_nn.head()

In [11]:
#data_orig = pd.concat([data_nn, data_dev], axis=0)

In [12]:
def get_pairs(data_orig, filter_length = MAX_LENGTH):

    big_str = list(data_orig.before.astype(str).values)
    output_list = list(data_orig.after.astype(str).values)
    types_list = list(data_orig['class'].values)
    
    stride = 3
    input_list = []
    pairs = []
    for i in range(len(big_str)):
        if big_str[i] != '<eos>':
            #print(big_str[i])
            cur_item = ['<norm>'] + getChars(big_str[i]) + ['</norm>']
            cur_type = types_list[i]
            #cur_item = ['<{}>'.format(cur_type)] + cur_item + ['</{}>'.format(cur_type)]
            #print(big_str[i-stride:i])
            prefix = getChars(' '.join(big_str[i-stride:i]))
            #print(prefix)
            prefix = ' '.join(prefix).split('< e o s >')[-1].split(' ')
            #print(prefix)
            suffix = getChars(' '.join(big_str[i+1:i+stride+1]))
            suffix = ' '.join(suffix).split('< e o s >')[0].split(' ')
            cur_item = prefix \
            + cur_item + \
            suffix

            cur_item = ' '.join(cur_item)
            cur_item = cur_item.replace('  ', ' ')
            cur_item = cur_item.replace('  ', ' ')
            if cur_item[0] == ' ':
                cur_item = cur_item[1:]
            pairs += [(cur_item, output_list[i], cur_type)]

    #pairs = list(zip(input_list, output_list))
    print('Len of pairs:', len(pairs))
    
    if filter_length:
        pairs = filterPairs(pairs)
        print('Len of pairs after filtering:', len(pairs))
    return pairs

In [13]:
list(range(3))[-4:2]

[0, 1]

In [14]:
train_pairs = get_pairs(data_learn)
dev_pairs = get_pairs(data_dev)

Len of pairs: 8319218
Len of pairs after filtering: 6885865
Len of pairs: 5547039
Len of pairs after filtering: 4592529


In [15]:
train_pairs[:10]

[('<norm> Д р у г а я </norm> в е р с и я б е р е т н а ч а л о',
  '<self>',
  'PLAIN'),
 ('<norm> в е р с и я </norm> б е р е т н а ч а л о о т', '<self>', 'PLAIN'),
 ('<norm> б е р е т </norm> н а ч а л о о т с л о в а', '<self>', 'PLAIN'),
 ('Д р у г а я в е р с и я б е р е т <norm> н а ч а л о </norm> о т с л о в а "',
  '<self>',
  'PLAIN'),
 ('в е р с и я б е р е т н а ч а л о <norm> о т </norm> с л о в а " н е п е я',
  '<self>',
  'PLAIN'),
 ('б е р е т н а ч а л о о т <norm> с л о в а </norm> " н е п е я "',
  '<self>',
  'PLAIN'),
 ('н а ч а л о о т с л о в а <norm> " </norm> н е п е я " -', 'sil', 'PUNCT'),
 ('о т с л о в а " <norm> н е п е я </norm> " - ч е л о в е к а',
  '<self>',
  'PLAIN'),
 ('с л о в а " н е п е я <norm> " </norm> - ч е л о в е к а в е д у щ е г о',
  'sil',
  'PUNCT'),
 ('" н е п е я " <norm> - </norm> ч е л о в е к а в е д у щ е г о т р е з в ы й',
  'sil',
  'VERBATIM')]

In [16]:
dev_pairs[:10]

[('<norm> П о </norm> с о с т о я н и ю н а 1 8 6 2 г о д', '<self>', 'PLAIN'),
 ('<norm> с о с т о я н и ю </norm> н а 1 8 6 2 г о д .', '<self>', 'PLAIN'),
 ('<norm> н а </norm> 1 8 6 2 г о д . ', '<self>', 'PLAIN'),
 ('П о с о с т о я н и ю н а <norm> 1 8 6 2 г о д </norm> . ',
  'тысяча восемьсот шестьдесят второй год',
  'DATE'),
 ('с о с т о я н и ю н а 1 8 6 2 г о д <norm> . </norm> ', 'sil', 'PUNCT'),
 ('<norm> О с н а щ а л и с ь </norm> л а т н ы м и р у к а в и ц а м и и',
  '<self>',
  'PLAIN'),
 ('О с н а щ а л и с ь <norm> л а т н ы м и </norm> р у к а в и ц а м и и с а б а т о н а м и',
  '<self>',
  'PLAIN'),
 ('О с н а щ а л и с ь л а т н ы м и <norm> р у к а в и ц а м и </norm> и с а б а т о н а м и с',
  '<self>',
  'PLAIN'),
 ('О с н а щ а л и с ь л а т н ы м и р у к а в и ц а м и <norm> и </norm> с а б а т о н а м и с н е',
  '<self>',
  'PLAIN'),
 ('л а т н ы м и р у к а в и ц а м и и <norm> с а б а т о н а м и </norm> с н е д л и н н ы м и',
  '<self>',
  'PLAIN'

In [17]:
input_lang, output_lang = Lang('nonnorm'), Lang('norm')

for pair in train_pairs + dev_pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])

In [18]:
#tst[tst.before != tst.after]

In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_output, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [20]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

In [21]:
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
   
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            
            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            
            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [22]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [23]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_output, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
        
        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words, decoder_attentions[:di + 1]

In [25]:
def trainIters(encoder, decoder, pairs, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, evaluate_each=False):
    start = time.time()
    plot_losses = []
    plot_accuracies = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
 
        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        if evaluate_each and iter % evaluate_each == 0 and iter != 0:
            preds = np.array([(item[1], ' '.join(evaluate(encoder, decoder, item[0])[0][:-1]), item[0]) for item in test_pairs])
            results = np.array([item[0] == item[1] for item in preds])
            print('\t\t eval accuracy: {:.3f}'.format(results.mean()))
            plot_accuracies.append(results.mean())

    showPlot(plot_losses)
    showPlot(plot_accuracies)
    
    return plot_losses

In [ ]:
torch.backends.cudnn.enabled = False
test_pairs = dev_pairs[:10000]


hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, n_layers=2)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,
                           n_layers =2, dropout_p=0.1)

if use_cuda:
    print('Using CUDA')
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

callback_num = 10000
plot_losses = trainIters(encoder1, attn_decoder1, train_pairs, 2000000, print_every=callback_num, 
                         plot_every=callback_num, evaluate_each=50000)

Using CUDA


In [ ]:
ind_to_test = -51

print(input_list[ind_to_test])
evaluate(encoder1, attn_decoder1, input_list[ind_to_test])[0]

In [187]:
test_pairs = dev_pairs[-2000:]


preds = np.array([(item[1], ' '.join(evaluate(encoder1, attn_decoder1, item[0])[0][:-1]), item[0]) for item in test_pairs])

In [188]:
results = np.array([item[0] == item[1] for item in preds] )

In [189]:
results.mean()

0.61099999999999999

In [190]:
results_norm = np.array([item[0].lower() == item[1].lower() for item in preds] )
results_norm.mean()

0.61099999999999999

In [192]:
preds[~results][0]

array(['пятьдесят один', 'пятьсот одиннадцать',
       'и с т р е б и т е л и Р - <CARDINAL> 5 1 </CARDINAL> « М у с т а н г »'],
      dtype='<U141')

In [ ]:
ind_to_test = 5

print(input_list[ind_to_test])
evaluate(encoder1, attn_decoder1, input_list[ind_to_test])[0]

In [ ]:
input_list[0]